In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD,Adadelta,Adam
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg16 import VGG16,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [6]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))


nb_train_samples = 20000
nb_val_samples = 2424
image_size = (224,224)
batch_size = 64
class_num = 10

In [3]:
def image_preprocess(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    return img

In [4]:
def test_model(model,test_path,csv='sample_submission.csv'):
    columns = ['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    df = pd.DataFrame(columns=columns)
    
    file_list = os.listdir(test_path)
    for i,file in enumerate(file_list):
        test_data = image_preprocess(test_path + '/' + file)
        y_pred = model.predict(test_data,batch_size=1,verbose=0)
        y_pred = np.clip(y_pred,0.001,0.999)
        y_pred = y_pred[0].tolist()
        df.loc[i] = [file] + y_pred
        
    df.to_csv(csv,index=None)

In [7]:
train_datagen = image.ImageDataGenerator(
                                         shear_range=0.3,
                                         rotation_range=15,
                                         height_shift_range=0.3,
                                         width_shift_range=0.3,
                                         channel_shift_range=15,
                                         preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


In [8]:
vgg16_base_model = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))

x = vgg16_base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(class_num, activation='softmax')(x)

vgg16_model = Model(inputs=vgg16_base_model.input, outputs=x)

vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [9]:
for layer in vgg16_model.layers[:19]:
    layer.trainable = False

for i,layer in enumerate(vgg16_model.layers):
    print(i,layer.name,layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False
19 global_average_pooling2d_1 True
20 dense_1 True


In [10]:
vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.Adam(lr=1e-3),
                    metrics=['accuracy'])


vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[EarlyStopping(patience=3)])

vgg16_model.save('vgg16_GAP_fc.h5')

Epoch 1/10
312/312 [==============================] - 351s 1s/step - loss: 2.5861 - acc: 0.3465 - val_loss: 2.4524 - val_acc: 0.3826
Epoch 2/10
312/312 [==============================] - 351s 1s/step - loss: 1.1108 - acc: 0.6316 - val_loss: 2.2605 - val_acc: 0.4212
Epoch 3/10
312/312 [==============================] - 349s 1s/step - loss: 0.8024 - acc: 0.7317 - val_loss: 1.8183 - val_acc: 0.4932
Epoch 4/10
312/312 [==============================] - 348s 1s/step - loss: 0.6641 - acc: 0.7770 - val_loss: 1.8984 - val_acc: 0.4843
Epoch 5/10
312/312 [==============================] - 346s 1s/step - loss: 0.6108 - acc: 0.7973 - val_loss: 1.8822 - val_acc: 0.4932
Epoch 6/10
312/312 [==============================] - 347s 1s/step - loss: 0.5493 - acc: 0.8168 - val_loss: 1.9904 - val_acc: 0.4856


In [11]:
vgg16_model = load_model('vgg16_GAP_fc.h5')

for layer in vgg16_model.layers[:15]:
    layer.trainable = False
    
for layer in vgg16_model.layers[15:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                  EarlyStopping(patience=3)])

vgg16_model.save('vgg16_GAP_15.h5')

Epoch 1/20
312/312 [==============================] - 352s 1s/step - loss: 0.4056 - acc: 0.8777 - val_loss: 1.1964 - val_acc: 0.6816
Epoch 2/20
312/312 [==============================] - 349s 1s/step - loss: 0.0890 - acc: 0.9742 - val_loss: 1.5029 - val_acc: 0.7225
Epoch 3/20
312/312 [==============================] - 351s 1s/step - loss: 0.0545 - acc: 0.9842 - val_loss: 0.9003 - val_acc: 0.7953
Epoch 4/20
312/312 [==============================] - 349s 1s/step - loss: 0.0374 - acc: 0.9888 - val_loss: 0.9429 - val_acc: 0.7941
Epoch 5/20
312/312 [==============================] - 349s 1s/step - loss: 0.0326 - acc: 0.9897 - val_loss: 1.2567 - val_acc: 0.7496
Epoch 6/20
312/312 [==============================] - 348s 1s/step - loss: 0.0310 - acc: 0.9905 - val_loss: 1.5554 - val_acc: 0.6818


In [12]:
vgg16_model = load_model('vgg16_GAP_15.h5')

for layer in vgg16_model.layers[:11]:
    layer.trainable = False
    
for layer in vgg16_model.layers[11:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
                     optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy']) 
    
vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                  EarlyStopping(patience=3)])

vgg16_model.save('vgg16_GAP_11.h5')

Epoch 1/20
312/312 [==============================] - 354s 1s/step - loss: 0.1235 - acc: 0.9618 - val_loss: 0.6387 - val_acc: 0.8590
Epoch 2/20
312/312 [==============================] - 353s 1s/step - loss: 0.0538 - acc: 0.9849 - val_loss: 1.1193 - val_acc: 0.8110
Epoch 3/20
312/312 [==============================] - 354s 1s/step - loss: 0.0416 - acc: 0.9872 - val_loss: 0.6757 - val_acc: 0.8682
Epoch 4/20
312/312 [==============================] - 352s 1s/step - loss: 0.0412 - acc: 0.9875 - val_loss: 1.8145 - val_acc: 0.6530


In [13]:
vgg16_model = load_model('vgg16_GAP_11.h5')

for layer in vgg16_model.layers[:7]:
    layer.trainable = False
    
for layer in vgg16_model.layers[7:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
                     optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy']) 
    
vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                  EarlyStopping(patience=3)])

vgg16_model.save('vgg16_GAP_7.h5')

Epoch 1/20
312/312 [==============================] - 415s 1s/step - loss: 0.0677 - acc: 0.9791 - val_loss: 0.5176 - val_acc: 0.8826
Epoch 2/20
312/312 [==============================] - 410s 1s/step - loss: 0.0521 - acc: 0.9846 - val_loss: 1.2058 - val_acc: 0.7822
Epoch 3/20
312/312 [==============================] - 410s 1s/step - loss: 0.0354 - acc: 0.9887 - val_loss: 1.1315 - val_acc: 0.8203
Epoch 4/20
312/312 [==============================] - 409s 1s/step - loss: 0.0299 - acc: 0.9903 - val_loss: 0.4073 - val_acc: 0.8949
Epoch 5/20
312/312 [==============================] - 409s 1s/step - loss: 0.0348 - acc: 0.9902 - val_loss: 0.4891 - val_acc: 0.8924
Epoch 6/20
312/312 [==============================] - 409s 1s/step - loss: 0.0292 - acc: 0.9910 - val_loss: 0.9298 - val_acc: 0.8453
Epoch 7/20
312/312 [==============================] - 409s 1s/step - loss: 0.0264 - acc: 0.9921 - val_loss: 0.4291 - val_acc: 0.8970


In [14]:
vgg16_model = load_model('vgg16_GAP_7.h5')
    
for layer in vgg16_model.layers:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
                     optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy']) 
    
vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                  EarlyStopping(patience=3)])

vgg16_model.save('vgg16_GAP_all.h5')

Epoch 1/20
312/312 [==============================] - 659s 2s/step - loss: 0.0359 - acc: 0.9890 - val_loss: 0.9667 - val_acc: 0.8459
Epoch 2/20
312/312 [==============================] - 651s 2s/step - loss: 0.0363 - acc: 0.9882 - val_loss: 0.5878 - val_acc: 0.9225
Epoch 3/20
312/312 [==============================] - 650s 2s/step - loss: 0.0233 - acc: 0.9931 - val_loss: 0.5264 - val_acc: 0.8890
Epoch 4/20
312/312 [==============================] - 650s 2s/step - loss: 0.0298 - acc: 0.9916 - val_loss: 0.5523 - val_acc: 0.8890
Epoch 5/20
312/312 [==============================] - 649s 2s/step - loss: 0.0228 - acc: 0.9932 - val_loss: 0.5061 - val_acc: 0.8979
Epoch 6/20
312/312 [==============================] - 649s 2s/step - loss: 0.0215 - acc: 0.9945 - val_loss: 0.5676 - val_acc: 0.8949
Epoch 7/20
312/312 [==============================] - 648s 2s/step - loss: 0.0140 - acc: 0.9960 - val_loss: 0.4638 - val_acc: 0.9106
Epoch 8/20
312/312 [==============================] - 649s 2s/step - 

In [20]:
vgg16_model.compile(loss='categorical_crossentropy',
                     optimizer=optimizers.Adam(lr=1e-5),
                    metrics=['accuracy']) 
    
vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=5,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                  ])

Epoch 1/5
312/312 [==============================] - 638s 2s/step - loss: 0.0012 - acc: 0.9997 - val_loss: 0.6061 - val_acc: 0.9312
Epoch 2/5
312/312 [==============================] - 636s 2s/step - loss: 6.0686e-04 - acc: 0.9998 - val_loss: 0.5574 - val_acc: 0.9373
Epoch 3/5
312/312 [==============================] - 636s 2s/step - loss: 7.1654e-04 - acc: 0.9998 - val_loss: 0.6181 - val_acc: 0.9275
Epoch 4/5
312/312 [==============================] - 636s 2s/step - loss: 1.0961e-04 - acc: 0.9999 - val_loss: 0.5776 - val_acc: 0.9297
Epoch 5/5
312/312 [==============================] - 637s 2s/step - loss: 7.1533e-05 - acc: 1.0000 - val_loss: 0.5940 - val_acc: 0.9364


In [21]:
vgg16_model.save('vgg16_GAP_final.h5')

In [22]:
test_model(vgg16_model,test_path=test_path,csv='vgg16_GAP_submission.csv')